In [1]:
import pandas as pd
import os
from boto3 import client
import json

In [2]:
%load_ext dotenv
%dotenv

In [3]:
aws_access_key_id = os.environ['aws_access_key_id']
aws_secret_access_key = os.environ['aws_secret_access_key']
aws_session_token = os.environ['aws_session_token']

In [4]:
BUCKET = "merged-tweets"
KEY = "testing-sample/merged.json"
s3_client = client('s3',
                   aws_access_key_id=aws_access_key_id,
                   aws_secret_access_key=aws_secret_access_key,
                   aws_session_token=aws_session_token)

In [7]:
result = s3_client.get_object(Bucket=BUCKET, Key=KEY) 
data = json.loads(result["Body"].read().decode())

In [8]:
cols = ['created_at', 'id', 'full_text', 'geo', 'coordinates',
        'retweet_count', 'favorite_count', 'reply_count',
       'quote_count', 'favorited', 'retweeted', 'possibly_sensitive',
       'lang']
te = [t['raw_data'] for t in data]
df = pd.DataFrame(te)[cols]

# Dask

In [17]:
import dask.dataframe as dd
import dask_ml.feature_extraction.text

ModuleNotFoundError: No module named 'dask_ml'

In [10]:
from dask.distributed import Client

client = Client(n_workers=2, threads_per_worker=2, memory_limit="1GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46007,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:44503,Total threads: 2
Dashboard: http://127.0.0.1:45055/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:43583,


In [18]:
client.shutdown()

2022-11-06 08:49:07,827 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [11]:
pd.options.display.precision = 2
pd.options.display.max_rows = 10

In [14]:
dask_df = dd.from_pandas(df, npartitions=10)

In [16]:
dask_df

,created_at,id,full_text,geo,coordinates,retweet_count,favorite_count,reply_count,quote_count,favorited,retweeted,possibly_sensitive,lang
npartitions=10,,,,,,,,,,,,,
0,object,int64,object,object,object,int64,int64,int64,int64,bool,bool,object,object
100,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,...,...,...,...,...,...,...,...,...,...,...,...,...
999,...,...,...,...,...,...,...,...,...,...,...,...,...
